In [5]:
import blockulib

In [10]:
import torch

tablica = [
        torch.tensor([
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 1, 1, 0, 1],
            [1, 0, 0, 0, 1, 1, 0, 1, 1],
            [1, 0, 1, 0, 0, 0, 1, 1, 1],
            [0, 1, 1, 1, 0, 1, 0, 1, 0],
            [1, 0, 1, 0, 1, 0, 1, 0, 1],
            [0, 0, 0, 1, 1, 1, 1, 1, 1],
            [1, 0, 0, 0, 1, 1, 1, 1, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 1]
        ]),
        torch.tensor([
            [1, 0, 1, 0, 1, 0, 1, 1, 0],
            [0, 0, 0, 1, 0, 1, 1, 0, 0],
            [1, 0, 1, 0, 1, 0, 0, 1, 0],
            [0, 1, 0, 1, 0, 0, 1, 1, 0],
            [1, 1, 1, 0, 1, 0, 1, 0, 1],
            [0, 1, 0, 1, 0, 1, 1, 0, 0],
            [1, 0, 0, 0, 1, 0, 0, 0, 1],
            [1, 0, 0, 1, 1, 1, 0, 1, 1],
            [1, 1, 1, 0, 1, 0, 1, 0, 1]
        ]),
        torch.tensor([
            [1, 1, 1, 1, 1, 1, 1, 1, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 0],
            [1, 1, 0, 0, 1, 1, 0, 0, 1],
            [1, 1, 0, 0, 1, 1, 0, 0, 1],
            [0, 0, 1, 1, 1, 0, 1, 1, 0],
            [0, 0, 1, 1, 1, 0, 1, 1, 0],
            [1, 1, 0, 0, 1, 1, 0, 0, 1],
            [1, 1, 0, 0, 1, 1, 0, 0, 1],
            [0, 0, 1, 1, 1, 0, 1, 1, 0]
        ])
    ]

In [16]:
pos, ind = blockulib.possible_moves(tablica)
print(pos)
print(ind)

[[[0. 1. 0. ... 0. 0. 0.]
  [0. 1. 1. ... 1. 0. 1.]
  [1. 1. 0. ... 0. 1. 1.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 1. 0. 1.]
  [1. 1. 1. ... 0. 1. 1.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 1. 1. ... 1. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [1. 0. 1. ... 0. 1. 0.]
  ...
  [1. 0. 0. ... 0. 0. 1.]
  [1. 0. 0. ... 0. 1. 1.]
  [1. 1. 1. ... 1. 0. 1.]]

 ...

 [[1. 0. 1. ... 1. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [1. 0. 1. ... 0. 1. 0.]
  ...
  [1. 0. 0. ... 0. 0. 1.]
  [1. 0. 0. ... 0. 1. 1.]
  [1. 1. 1. ... 1. 0. 1.]]

 [[1. 0. 1. ... 1. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [1. 0. 1. ... 0. 1. 0.]
  ...
  [1. 0. 0. ... 0. 0. 1.]
  [1. 0. 0. ... 0. 1. 1.]
  [1. 1. 1. ... 1. 0. 1.]]

 [[1. 0. 1. ... 1. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [1. 0. 1. ... 0. 1. 0.]
  ...
  [1. 0. 0. ... 0. 0. 1.]
  [1. 0. 0. ... 0. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]]
tensor([0.